In [1]:
from gym.spaces import Box
import numpy as np

from gym.envs.classic_control import PendulumEnv

In [2]:
class StatelessPendulum(PendulumEnv):
    """Partially observable variant of the Pendulum gym environment.

    https://github.com/openai/gym/blob/master/gym/envs/classic_control/
    pendulum.py

    We delete the angular velocity component of the state, so that it
    can only be solved by a memory enhanced model (policy).
    """

    def __init__(self, config=None):
        config = config or {}
        g = config.get("g", 10.0)

        super().__init__(g=g)

        # Fix our observation-space (remove angular velocity component).
        high = np.array([1., 1.], dtype=np.float32)
        self.observation_space = Box(low=-high, high=high, dtype=np.float32)

    def step(self, action):
        next_obs, reward, done, info = super().step(action)
        # next_obs is [cos(theta), sin(theta), theta-dot (angular velocity)]
        return next_obs[:-1], reward, done, info

    def reset(self):
        init_obs = super().reset()
        # init_obs is [cos(theta), sin(theta), theta-dot (angular velocity)]
        return init_obs[:-1]


In [3]:
import numpy as np

import ray
from ray import tune
from ray.tune.registry import register_env
from ray.rllib.models import ModelCatalog


2021-12-09 14:27:54,611	WARNING deprecation.py:45 -- DeprecationWarning: `ray.rllib.utils.window_stat.WindowStat` has been deprecated. Use `ray.rllib.utils.metrics.window_stat.WindowStat` instead. This will raise an error in the future!


In [4]:
ray.init(num_cpus=0 or None)

2021-12-09 14:27:58,219	WARNING services.py:1838 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.


{'node_ip_address': '172.18.0.3',
 'raylet_ip_address': '172.18.0.3',
 'redis_address': '172.18.0.3:48762',
 'object_store_address': '/tmp/ray/session_2021-12-09_14-27-56_974321_985/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-12-09_14-27-56_974321_985/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2021-12-09_14-27-56_974321_985',
 'metrics_export_port': 56289,
 'node_id': 'd6b5b0d47bcfc03d7e51529ccf9ca44cd3765ceefb7350068da1e0c4'}

In [6]:
register_env("StatelessPendulum", lambda _: StatelessPendulum())


In [7]:
env = StatelessPendulum()

In [8]:
env.reset()

array([ 0.07319583, -0.9973176 ], dtype=float32)

In [19]:
next_obs, reward, done, info = env.step([0.1])

In [20]:
reward

-2.2835522176273275

In [13]:
configs = {
    "PPO": {
        "num_sgd_iter": 5,
        "sgd_minibatch_size": 128, 
        "simple_optimizer": True,
        "model": {
            "vf_share_layers": True,
        },
        "vf_loss_coeff": 0.0001,
    },
    "IMPALA": {
        "num_workers": 2,
        "num_gpus": 0,
        "vf_loss_coeff": 0.01,
    },
}

In [14]:
config = dict(
    configs['PPO'],
    **{
        "env": 'StatelessPendulum',
        # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.
        "num_gpus": 0,
        "model": {
            "use_lstm": True,
            "lstm_cell_size": 256,
            "lstm_use_prev_action": True,
            "lstm_use_prev_reward": True,
        },
        "framework": 'torch',
    })

In [15]:
stop = {
    "training_iteration": 40,
    "timesteps_total": 100000,
    "episode_reward_mean": 90.,
}

In [16]:
results = tune.run('PPO', config=config, stop=stop, verbose=2)

2021-12-09 14:34:59,620	WARNING callback.py:114 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


(bundle_reservation_check_func pid=1107) 2021-12-09 14:35:01,294	WARNING deprecation.py:45 -- DeprecationWarning: `ray.rllib.utils.window_stat.WindowStat` has been deprecated. Use `ray.rllib.utils.metrics.window_stat.WindowStat` instead. This will raise an error in the future!
(PPO pid=1107) 2021-12-09 14:35:01,632	WARNING deprecation.py:45 -- DeprecationWarning: `simple_optimizer` has been deprecated. This will raise an error in the future!
(PPO pid=1107) 2021-12-09 14:35:01,632	INFO trainer.py:719 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=1108) 2021-12-09 14:35:03,183	WARNING deprecation.py:45 -- DeprecationWarning: `ray.rllib.utils.window_stat.WindowStat` has been deprecated. Use `ray.rllib.utils.metrics.window_stat.WindowStat` instead. This will raise an error in the future!
(pid=1115) 2021-12-09 14:35:03,272	WARNING deprecation.py:45 -- DeprecationWarning: `ray.rllib.utils.window_stat.WindowStat` has bee

(PPO pid=1107) 2021-12-09 14:35:07,440	WARNING deprecation.py:45 -- DeprecationWarning: `_get_slice_indices` has been deprecated. This will raise an error in the future!
(PPO pid=1107) 2021-12-09 14:35:07,443	WARNING deprecation.py:45 -- DeprecationWarning: `slice` has been deprecated. Use `SampleBatch[start:stop]` instead. This will raise an error in the future!


Trial PPO_StatelessPendulum_ce305_00000 reported episode_reward_max=nan,episode_reward_min=nan,episode_reward_mean=nan,episode_len_mean=nan,episode_media={},episodes_this_iter=0,policy_reward_min={},policy_reward_max={},policy_reward_mean={},custom_metrics={},sampler_perf={},off_policy_estimator={},num_healthy_workers=2,timesteps_this_iter=0,agent_timesteps_total=4000,timers={'sample_time_ms': 3829.51, 'sample_throughput': 1044.52, 'learn_time_ms': 4320.37, 'learn_throughput': 925.847, 'update_time_ms': 2.478},info={'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'cur_kl_coeff': 0.19999999999999996, 'cur_lr': 5e-05, 'total_loss': 11.967570070786909, 'policy_loss': 0.025624675382718894, 'vf_loss': 119400.87341382576, 'vf_explained_var': 0.0015878088546521737, 'kl': 0.009291289999347379, 'entropy': 1.2064687035300514, 'entropy_coeff': 0.0}, 'model': {}, 'custom_metrics': {}}}, 'num_steps_sampled': 4000, 'num_agent_steps_sampled': 4000, 'num_steps_trained': 4000

Trial PPO_StatelessPendulum_ce305_00000 reported episode_reward_max=nan,episode_reward_min=nan,episode_reward_mean=nan,episode_len_mean=nan,episode_media={},episodes_this_iter=0,policy_reward_min={},policy_reward_max={},policy_reward_mean={},custom_metrics={},sampler_perf={},off_policy_estimator={},num_healthy_workers=2,timesteps_this_iter=0,agent_timesteps_total=8000,timers={'sample_time_ms': 6100.037, 'sample_throughput': 655.734, 'learn_time_ms': 4252.824, 'learn_throughput': 940.551, 'update_time_ms': 2.363},info={'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'cur_kl_coeff': 0.19999999999999996, 'cur_lr': 5e-05, 'total_loss': 12.090836409366492, 'policy_loss': 0.035774418372999536, 'vf_loss': 120528.66586174243, 'vf_explained_var': 0.004245922059723825, 'kl': 0.010978210507715726, 'entropy': 1.2338258078604034, 'entropy_coeff': 0.0}, 'model': {}, 'custom_metrics': {}}}, 'num_steps_sampled': 8000, 'num_agent_steps_sampled': 8000, 'num_steps_trained': 800

...

Trial PPO_StatelessPendulum_ce305_00000 reported episode_reward_max=nan,episode_reward_min=nan,episode_reward_mean=nan,episode_len_mean=nan,episode_media={},episodes_this_iter=0,policy_reward_min={},policy_reward_max={},policy_reward_mean={},custom_metrics={},sampler_perf={},off_policy_estimator={},num_healthy_workers=2,timesteps_this_iter=0,agent_timesteps_total=92000,timers={'sample_time_ms': 8068.821, 'sample_throughput': 495.735, 'learn_time_ms': 4166.963, 'learn_throughput': 959.932, 'update_time_ms': 2.191},info={'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'cur_kl_coeff': 0.0031249999999999993, 'cur_lr': 5e-05, 'total_loss': 5.457133235353412, 'policy_loss': 0.024555991623889318, 'vf_loss': 54325.66075994318, 'vf_explained_var': -7.848667375969165e-06, 'kl': 0.003624000229419886, 'entropy': 0.9445365515622226, 'entropy_coeff': 0.0}, 'model': {}, 'custom_metrics': {}}}, 'num_steps_sampled': 92000, 'num_agent_steps_sampled': 92000, 'num_steps_trained'

Trial PPO_StatelessPendulum_ce305_00000 reported episode_reward_max=nan,episode_reward_min=nan,episode_reward_mean=nan,episode_len_mean=nan,episode_media={},episodes_this_iter=0,policy_reward_min={},policy_reward_max={},policy_reward_mean={},custom_metrics={},sampler_perf={},off_policy_estimator={},num_healthy_workers=2,timesteps_this_iter=0,agent_timesteps_total=96000,timers={'sample_time_ms': 8054.071, 'sample_throughput': 496.643, 'learn_time_ms': 4168.537, 'learn_throughput': 959.569, 'update_time_ms': 2.172},info={'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'cur_kl_coeff': 0.0015624999999999997, 'cur_lr': 5e-05, 'total_loss': 5.4939002932924215, 'policy_loss': 0.024137740373385674, 'vf_loss': 54697.492199337124, 'vf_explained_var': 2.9766920841101442e-05, 'kl': 0.008610114820036022, 'entropy': 0.9873132958556666, 'entropy_coeff': 0.0}, 'model': {}, 'custom_metrics': {}}}, 'num_steps_sampled': 96000, 'num_agent_steps_sampled': 96000, 'num_steps_traine

Trial PPO_StatelessPendulum_ce305_00000 reported episode_reward_max=nan,episode_reward_min=nan,episode_reward_mean=nan,episode_len_mean=nan,episode_media={},episodes_this_iter=0,policy_reward_min={},policy_reward_max={},policy_reward_mean={},custom_metrics={},sampler_perf={},off_policy_estimator={},num_healthy_workers=2,timesteps_this_iter=0,agent_timesteps_total=100000,timers={'sample_time_ms': 8054.903, 'sample_throughput': 496.592, 'learn_time_ms': 4172.287, 'learn_throughput': 958.707, 'update_time_ms': 2.167},info={'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'cur_kl_coeff': 0.0015624999999999997, 'cur_lr': 5e-05, 'total_loss': 5.3703459450692845, 'policy_loss': 0.01972379618409005, 'vf_loss': 53506.06985085227, 'vf_explained_var': -3.84583617701675e-05, 'kl': 0.00980129351949309, 'entropy': 0.8882211869413202, 'entropy_coeff': 0.0}, 'model': {}, 'custom_metrics': {}}}, 'num_steps_sampled': 100000, 'num_agent_steps_sampled': 100000, 'num_steps_trained

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPO_StatelessPendulum_ce305_00000,TERMINATED,172.18.0.3:1107,25,200.292,100000,nan,nan,nan,nan


2021-12-09 14:38:24,287	INFO tune.py:626 -- Total run time: 204.67 seconds (204.50 seconds for the tuning loop).
